In [1]:
import os
import sys
from pathlib import Path

sys.path.append('..')


In [2]:
# Parameters for development:
SEED_DEV = 0
DATASET = 'MNIST'
PATH_ROOT = Path(os.getcwd()).absolute().parent
PATH_CHECKPOINT = os.path.join(PATH_ROOT, 'pretrained_clf', 'mnist_cnn.ckpt')
PATH_DATA_CLEAN = os.path.join(PATH_ROOT, 'results', 'exp1234', 'MNIST', 'AdvClean.n_100.pt')
PATH_DATA_ADV = os.path.join(PATH_ROOT, 'results', 'exp1234', 'MNIST', 'FGSM.Linf.n_100.e_0.28.pt')

print(PATH_ROOT)

/home/lukec/workspace/baard_v4


In [3]:
import torch 
from torch import Tensor

from baard.classifiers import DATASETS, MNIST_CNN
from baard.utils.torch_utils import dataset2tensor

In [4]:
my_model = MNIST_CNN.load_from_checkpoint(PATH_CHECKPOINT)

In [5]:
loader_train = my_model.train_dataloader()

print('N_BATCHES:', len(loader_train))

N_BATCHES: 235


In [6]:
out_x = torch.ones([1, 10])
out_x

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [7]:
out_neighbors = torch.rand([100, 10])
out_neighbors[:5]

tensor([[0.3838, 0.9344, 0.2960, 0.1121, 0.7724, 0.5019, 0.2902, 0.6124, 0.5415,
         0.4882],
        [0.2683, 0.3230, 0.1970, 0.7373, 0.3190, 0.8449, 0.2340, 0.6253, 0.5500,
         0.8823],
        [0.7077, 0.6314, 0.2260, 0.9094, 0.3002, 0.2239, 0.7439, 0.7130, 0.1006,
         0.4893],
        [0.8219, 0.8388, 0.6175, 0.4072, 0.2304, 0.3778, 0.8319, 0.2559, 0.0707,
         0.4168],
        [0.4078, 0.3945, 0.6803, 0.3790, 0.5618, 0.5245, 0.9495, 0.1513, 0.4839,
         0.1241]])

In [8]:
pairwise_dist = torch.pairwise_distance(out_x, out_neighbors, p=2)
pairwise_dist

tensor([1.7590, 1.7716, 1.7716, 1.8222, 1.8384, 1.7720, 2.0014, 1.6906, 2.0201,
        2.2896, 1.7390, 2.1076, 2.0130, 1.5232, 1.5277, 1.5928, 1.5179, 1.2507,
        1.7426, 1.7740, 1.8373, 1.5460, 1.8588, 1.3966, 1.7502, 2.2568, 2.2418,
        1.8251, 1.5505, 1.6906, 1.3846, 2.0466, 1.0446, 1.6668, 1.6220, 1.8653,
        1.7152, 1.9763, 1.8049, 1.5903, 1.8361, 2.0875, 1.7095, 1.4850, 2.0638,
        1.8683, 2.1689, 1.3711, 2.0118, 1.9425, 1.8881, 1.6111, 2.1733, 1.7556,
        2.2239, 2.1002, 1.7577, 1.3280, 1.6377, 2.3754, 1.6075, 1.9305, 2.2990,
        1.7156, 2.0188, 2.0575, 2.1548, 1.6505, 1.3199, 2.3857, 1.9721, 2.0631,
        1.9617, 2.0254, 1.6487, 1.6255, 1.6487, 1.5742, 1.9877, 1.8489, 1.8782,
        1.9665, 2.1459, 1.7961, 1.8274, 1.5183, 2.0978, 1.5085, 1.8756, 2.1728,
        2.2192, 1.9371, 1.3039, 2.0415, 1.9992, 2.1436, 1.5154, 1.3798, 2.0325,
        1.6883])

In [9]:
def get_MLE_LID(x: Tensor, batch: Tensor, k: int) -> float:
    pairwise_dist = torch.pairwise_distance(x, batch, p=2)
    k_dist, k_indices = torch.topk(pairwise_dist, k=k, sorted=True, largest=True)
    max_dist = k_dist[0]  # 1st is the largest.
    lid = - len(k_dist) / torch.log(k_dist / max_dist).sum()
    return lid.item()

In [10]:
result = get_MLE_LID(out_x, out_neighbors, 20)
result

11.470376968383789

In [11]:
a = torch.rand((1, 1, 28, 28))
b = torch.rand((1, 1, 28, 28))
c = torch.rand((1, 1, 28, 28))

torch.vstack([a, b, c]).size()

torch.Size([3, 1, 28, 28])

In [12]:
t = torch.tensor([
    [1.,2.,3.],
    [4.,5.,6.],
    [7., 8., 9.],
])
print("Tensor:", t)

Tensor: tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])


In [13]:
import torch.nn.functional as F

t1 = F.normalize(t, p=1.0, dim = 1)
t2 = F.normalize(t, p=2.0, dim = 0)

print("Normalized tensor:", t1)
print("Normalized tensor:", t2)

Normalized tensor: tensor([[0.1667, 0.3333, 0.5000],
        [0.2667, 0.3333, 0.4000],
        [0.2917, 0.3333, 0.3750]])
Normalized tensor: tensor([[0.1231, 0.2074, 0.2673],
        [0.4924, 0.5185, 0.5345],
        [0.8616, 0.8296, 0.8018]])


In [14]:
t1.mean(1)

tensor([0.3333, 0.3333, 0.3333])

In [15]:
t2.mean(1)

tensor([0.1992, 0.5151, 0.8310])

In [16]:
def min_max_normalize(X: Tensor, dim=0) -> Tensor:
    """Apply min-max normalization."""
    X_min = torch.min(X, dim=dim)[0].unsqueeze(dim=dim)
    X_max = torch.max(X, dim=dim)[0].unsqueeze(dim=dim)
    X_norm = (X - X_min) / (X_max - X_min)
    return X_norm

In [17]:
min_max_normalize(t)

tensor([[0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.5000],
        [1.0000, 1.0000, 1.0000]])

In [18]:
X_min = torch.min(t, dim=1)[0].unsqueeze(dim=1)
X_max = torch.max(t, dim=1)[0].unsqueeze(dim=1)


In [19]:
print(X_min)
print(X_max)

tensor([[1.],
        [4.],
        [7.]])
tensor([[3.],
        [6.],
        [9.]])


In [20]:
x = torch.arange(1., 6.)
torch.topk(x, 3, largest=False, sorted=True)

torch.return_types.topk(
values=tensor([1., 2., 3.]),
indices=tensor([0, 1, 2]))

In [21]:
X_min.unsqueeze(dim=1)

tensor([[[1.]],

        [[4.]],

        [[7.]]])

In [22]:
import sklearn.preprocessing as P

scaler = P.MinMaxScaler((0, 1))
scaler.fit_transform(t)

array([[0. , 0. , 0. ],
       [0.5, 0.5, 0.5],
       [1. , 1. , 1. ]])

In [24]:
import numpy as np

def get_MLE_LID(x: Tensor, batch: Tensor, k: int) -> Tensor:
    """Compute one sample Maximum Likelihood Estimator of LID within k nearest neighbors.
    It the batch contains x itself, it returns LID from k-1 neighbors.
    """
    pairwise_dist = torch.pairwise_distance(x, batch, p=2)
    # Nearest neighbors = smallest distance
    k_dist, _ = torch.topk(pairwise_dist, k=k, sorted=True, largest=False)  # Returns (values, indices)
    if np.isclose(k_dist[0].item(), 0):  # Find x itself
        k_dist = k_dist[1:]  # Find k+1 and discard itself.
    max_dist = k_dist[-1]  # the max neighbor distance
    lid = - len(k_dist) / torch.log(k_dist / max_dist).sum()
    return lid

In [27]:
a = torch.zeros((1, 10))
a_neighbors = torch.rand((100, 10))
k = 20
get_MLE_LID(a, a_neighbors, k)

tensor(10.8017)

In [28]:
b = torch.zeros((1, 1000))
b_neighbors = torch.rand((100, 1000))
k = 20
get_MLE_LID(b, b_neighbors, k)

tensor(108.5023)

In [30]:
t_d1 = torch.nn.functional.normalize(t, p=2.0, dim=1)
t_d1

tensor([[0.2673, 0.5345, 0.8018],
        [0.4558, 0.5698, 0.6838],
        [0.5026, 0.5744, 0.6462]])

In [31]:
t_d0 = torch.nn.functional.normalize(t, p=2.0, dim=0)
t_d0

tensor([[0.1231, 0.2074, 0.2673],
        [0.4924, 0.5185, 0.5345],
        [0.8616, 0.8296, 0.8018]])